In [1]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 

     |████████████████████████████████| 592.3MB 90.0MB/s 
ERROR: torchvision 0.4.2 has requirement torch==1.3.1, but you'll have torch 0.3.0.post4 which is incompatible.
ERROR: fastai 1.0.59 has requirement torch>=1.0.0, but you'll have torch 0.3.0.post4 which is incompatible.
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3.1:
      Successfully uninstalled torch-1.3.1


In [2]:
!pip3 install torchvision

  Using cached https://files.pythonhosted.org/packages/88/95/90e8c4c31cfc67248bf944ba42029295b77159982f532c5689bcfe4e9108/torch-1.3.1-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: torch 0.3.0.post4
    Uninstalling torch-0.3.0.post4:
      Successfully uninstalled torch-0.3.0.post4


In [0]:
import numpy as np
import torch
import torch.nn as nn # nn class becomes the parent class later when defining the model
import torch.nn.functional as F # functional class consists of various methods of loss functions
import torch.utils.data as data # Used to load dataset
import torchvision
from torchvision import transforms # used to perform various transformations on the input image
import matplotlib.pyplot as plt 
import torch.optim as optim

In [0]:
# transforms.Compose - it composes several transforms together
transform_img = transforms.Compose([
    transforms.Resize((300, 300)), # resized to 100 * 100
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],   # input[channel] = (input[channel] - mean[channel]) / std[channel]
                        std=[0.5, 0.5, 0.5])])

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
class MyNetwork(nn.Module):
    # Class MyNetwork will inherit nn.Module
    def __init__(self):
        super().__init__();
        # super().__init__() will call the constructor of nn.Module. The parameters of the methods
        # that we will use later (of nn class) will then get initialized
        self.conv1 = nn.Conv2d(3, 16, 5); # where 5 is the size of the filter, 3 - input channels, 6 - o/p channels
        self.pool = nn.MaxPool2d(2, 2);
        self.conv2 = nn.Conv2d(16, 18, 20);
        self.conv3 = nn.Conv2d(18, 20, 30);
        self.conv4 = nn.Conv2d(20, 16, 5);
        self.fc1 = nn.Linear(6 * 6 * 16, 1200); # Input units, o/p units in layer
        self.fc2 = nn.Linear(1200, 100);
        self.fc3 = nn.Linear(100, 50);
        self.fc4 = nn.Linear(50, 5);
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)));
        x = self.pool(F.relu(self.conv2(x)));
        x = self.pool(F.relu(self.conv3(x)));
        x = self.pool(F.relu(self.conv4(x)));
        x = x.view(-1, 6 * 6 * 16); # in pytorch, images are of the form [number of channels, height, width]
        x = F.relu(self.fc1(x));
        x = F.relu(self.fc2(x));
        x = F.relu(self.fc3(x));
        x = self.fc4(x);
        return x
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"); # use gpu if gpu is available else use cpu
mynetwork = MyNetwork()
mynetwork.to(device)

MyNetwork(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 18, kernel_size=(20, 20), stride=(1, 1))
  (conv3): Conv2d(18, 20, kernel_size=(30, 30), stride=(1, 1))
  (conv4): Conv2d(20, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=1200, bias=True)
  (fc2): Linear(in_features=1200, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=5, bias=True)
)

In [0]:
train_data_path = "/content/gdrive/My Drive/Flower Classification/train"
test_data_path = "/content/gdrive/My Drive/Flower Classification/test"

In [0]:
train_data = torchvision.datasets.ImageFolder(root = train_data_path, transform = transform_img)
# num_workers means the number of sub-processes employed in order to load the data. Optimum - 2, 4
train_data_loader = data.DataLoader(train_data, batch_size = 32, shuffle=True,  num_workers=4)

test_data = torchvision.datasets.ImageFolder(root = test_data_path, transform = transform_img)
test_data_loader  = data.DataLoader(test_data, batch_size = 32, shuffle=True, num_workers=4) 

In [0]:
# Here we define which type of loss function we'll use and what type of optimization we will do
criterion = nn.CrossEntropyLoss() # -ylog(y_) for softmax
optimizer = optim.Adam(mynetwork.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False) # Momentum is generally taken as 0.9
# When net.parameters() is called, automatically requires_grad is set True for the parameters in order to calculate 
# their gradients

In [10]:
for epoch in range(20): 
    for i, data in enumerate(train_data_loader):
        inputs, labels = data;
        inputs, labels = inputs.to(device), labels.to(device);
        optimizer.zero_grad(); # Zero the parameter gradeints so that in the next cycle, those values don't create
        # trouble
        outputs = mynetwork.forward(inputs); # Passes inputs to the forward method inside MyNetwork class
        loss = criterion(outputs, labels);
        loss.backward(); # Tells that we have to calculate the gradient of Loss w.r.t the parameters and the gradients
        # are stored in the .grad property of the parameters
        optimizer.step(); # This leads to parameter update
        print(loss.item()); # Prints the scalar value of loss
    # As the training data is divided into mini-batches, per 32 inputs,
    # the cost will be calculated and then updated.

1.6297695636749268
1.6016563177108765
1.7440749406814575
1.6196823120117188
1.6201727390289307
1.6227295398712158
1.6147724390029907
1.5981509685516357
1.5955283641815186
1.6203199625015259
1.6079390048980713
1.608791470527649
1.6267565488815308
1.6062688827514648
1.6306995153427124
1.6034427881240845
1.5959265232086182
1.587576150894165
1.593961238861084
1.5807892084121704
1.6136062145233154
1.5884678363800049
1.4975444078445435
1.6186305284500122
1.7879447937011719
1.636022686958313
1.5717169046401978
1.5942227840423584
1.5860028266906738
1.5898098945617676
1.59927237033844
1.581847906112671
1.5933680534362793
1.6027698516845703
1.6065587997436523
1.6039183139801025
1.6084507703781128
1.5889081954956055
1.5970067977905273
1.5679720640182495
1.5938390493392944
1.571629285812378
1.5423344373703003
1.5724763870239258
1.7163208723068237
1.4856822490692139
1.553397297859192
1.5361913442611694
1.4454272985458374
1.6892955303192139
1.6106053590774536
1.536486029624939
1.4823635816574097
1.4

In [0]:
def corr(x): # To get proper correspondence between the outputs and the labels
    x = x.detach().numpy(); # Detach() was used as one can't convert a pytorch tensor to a numpy array if
    # required_grad is set True for that variable
    x = x.argmax(axis = 1);
    return x;

In [16]:
correct = 0;
total = 0;
for i, data in enumerate(test_data_loader):
    inputs, labels = data;
    inputs, labels = inputs.to(device), labels;
    outputs = mynetwork.forward(inputs);
    outputs = outputs.cpu();
    outputs = corr(outputs);
    labels = labels.detach().numpy();
    for i in range(len(outputs)):
        total = total + 1;
        if outputs[i] == labels[i]:
            correct = correct + 1;
print("Hence, the test set accuracy is ", (correct/total) * 100);

Hence, the test set accuracy is  44.69387755102041


In [0]:
inputs.cuda();
print(mynetwork);

MyNetwork(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 18, kernel_size=(20, 20), stride=(1, 1))
  (conv3): Conv2d(18, 20, kernel_size=(30, 30), stride=(1, 1))
  (conv4): Conv2d(20, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=1200, bias=True)
  (fc2): Linear(in_features=1200, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=5, bias=True)
)


True
